In [21]:
import os, psutil, gc
import json, pickle
import time
import pprint
import copy 

from collections import defaultdict
import random
import numpy as np

from sklearn.decomposition import PCA


In [3]:
# base_dir
base_dir = '/groups/kjun/tnn/datasets/'

# dataset path
data_dir = base_dir + "/prm800k/math_splits"
pca_dir = base_dir + "/completion_embeds"

# llm and prm path
llm_dir = base_dir + "/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.Q4_K_M.gguf"
prm_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data-GGUF/Llama3.1-8B-PRM-Deepseek-Data.Q4_K_M.gguf"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"

In [44]:
level = 4
config_name = f"bon--n-256--level-{level}--train--v01--chunk-0_200--trial-0"
# with open(f"{pca_dir}/{config_name}.pkl", "rb") as fin:
#     dataset = pickle.load(fin)
    
dataset = []
for level in range(1, 6):
    config_name = f"bon--n-256--level-{level}--train--v01--chunk-0_200--trial-0"
    with open(f"{pca_dir}/{config_name}.pkl", "rb") as fin:
        dataset += pickle.load(fin)

In [40]:
print(len(dataset))

2029969


In [ ]:
config_name = f"bon--n-256--level-all--train--v01--chunk-0_200--trial-0"
def train_pca(dataset, config_name, ndim):
    _all_embeds = []
    for data in dataset:
        if data['is_completed'] == 0:
            _all_embeds.append(data['embeds'])
    all_embeds = np.vstack(_all_embeds)
    pca = PCA(n_components = ndim)
    embeds_mean = np.mean(all_embeds, axis=0)
    all_embed_centered = all_embeds - embeds_mean
    pca.fit(all_embed_centered)
    
    with open(f"results/{config_name}--pca-{ndim}.pkl", "wb") as fout:
        save_data = [embeds_mean, pca.components_]
        pickle.dump(save_data, fout)

ndim = 256
train_pca(dataset, config_name, ndim)

In [19]:

print(all_embeds.shape)

(495912, 2048)


In [22]:
ndim = 256
pca = PCA(n_components = ndim)

In [24]:
embeds_mean = np.mean(all_embeds, axis=0)
all_embed_centered = all_embeds - embeds_mean
pca.fit(all_embed_centered)
print(pca.components_)

[[ 0.02022387  0.03108055 -0.00753227 ... -0.03004597  0.01598934
  -0.02011937]
 [ 0.0200042   0.02868506  0.01134786 ...  0.00494341  0.06565451
   0.02841046]
 [-0.00098525 -0.02191211 -0.01001046 ... -0.01901794 -0.00709826
   0.00114151]
 ...
 [ 0.03349313 -0.02696786 -0.01541481 ...  0.02019269  0.0139631
   0.0024994 ]
 [-0.01077085 -0.04135703  0.03308401 ...  0.05029943  0.0230169
  -0.00894882]
 [-0.02573958 -0.02332509  0.00326485 ...  0.0545907  -0.03538265
   0.04684769]]


In [25]:
pca.components_.shape

(256, 2048)

In [ ]:
all_embed_transformed = pca.fit_transform(all_embed_centered)
print(all_embed_transformed[:1,:10])

In [32]:
print(all_embed_transformed[:1,:10])
print(pca.components_)

[[-12.884256   42.954163    3.8596764  24.579433    9.634513  -13.448514
   -3.5539434 -11.564653  -11.707997    9.229043 ]]
[[ 0.02022384  0.03108057 -0.00753224 ... -0.03004593  0.01598937
  -0.02011932]
 [ 0.0200042   0.02868508  0.01134786 ...  0.00494342  0.06565449
   0.02841049]
 [-0.00098524 -0.02191202 -0.01001049 ... -0.01901795 -0.00709834
   0.00114146]
 ...
 [ 0.01900635  0.02285737 -0.00608761 ... -0.00576427 -0.02189744
  -0.00754477]
 [ 0.02213326  0.02206101 -0.02618645 ... -0.00096279  0.04043048
   0.00293596]
 [-0.00328571  0.01436644  0.01008494 ... -0.00529047  0.02228228
   0.01003818]]


In [34]:
all_embed_transformed_2 = np.matmul(all_embed_centered, pca.components_.T)

In [35]:
print(all_embed_transformed_2[:1,:10])

[[-12.884413   42.95352     3.8605475  24.57958     9.634318  -13.449513
   -3.5537057 -11.5649185 -11.707938    9.229224 ]]


In [36]:
with open(f"{pca_dir}/{config_name}-pca.pkl", "wb") as fout:
    save_data = [embeds_mean, pca.components_]
    pickle.dump(save_data, fout)

In [ ]:
pca = PCA(n_comp